In [23]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import sys
import pandas as pd

import tensorflow as tf

DATA_PATH = "./data/TRAIN.csv"
BATCH_SIZE = 1
N_COLUMNS = 38
TRAIN_STEPS = 3
N_SAMPLES = 5

def batch_generator(filename):
    filename_queue = tf.train.string_input_producer(filename)
    reader = tf.TextLineReader(skip_header_lines=1)
    _, value = reader.read(filename_queue)

    record_defaults = [[1.0] for _ in range(N_COLUMNS)]

    # read in batch rows of data
    content = tf.decode_csv(value, record_defaults=record_defaults)

    #pack all 37 features into a tensor
    features = tf.stack(content[:N_COLUMNS-1])

    #last column to label
    label = content[-1]

    min_after_dequeue = 20

    capacity = 21

    data_batch, label_batch = tf.train.shuffle_batch([features, label], batch_size=BATCH_SIZE, capacity=capacity, min_after_dequeue=min_after_dequeue)
    return data_batch, label_batch

def generate_batches(data_batch, label_batch):
    with tf.Session() as sess:
        try:
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(coord=coord)
            for _ in range(TRAIN_STEPS):
                features, labels = sess.run([data_batch, label_batch])
                print(features," + ", labels)

        except tf.errors.OutOfRangeError:
            print('Done training -- limit reached')
        finally:
            coord.request_stop()
            coord.join(threads)

#define placehodelrs
X = tf.placeholder(tf.float32,name="X")
Y = tf.placeholder(tf.float32,name="Y")

#create weight and bias
W = tf.Variable(0.0, name="weight", dtype=tf.float32)
b = tf.Variable(0.0, name="bias", dtype=tf.float32)

#Predict
Y_pred = X*W + b

#loss
loss = tf.square(Y-Y_pred, name="loss")

optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

with tf.Session() as sess:
    try:
        sess.run(tf.global_variables_initializer())
        writer = tf.summary.FileWriter("./graphs/linearReg", sess.graph)
        for i in range(TRAIN_STEPS):
            total_loss = 0.0
            data_batch, label_batch = batch_generator([DATA_PATH])
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(coord=coord)
            for _ in range(N_SAMPLES):
                features, labels = sess.run([data_batch, label_batch])
                print(features, labels)
                _, loss_per_step = sess.run([optimizer, loss], feed_dict={X:features, Y:labels})
                total_loss += loss_per_step
    except tf.errors.OutOfRangeError:
        print('Done training -- limit reached')
    finally:
            coord.request_stop()
            coord.join(threads)
            
            print("Epoca {0}: {1}".format(i, total_loss/N_SAMPLES))
            writer.close()

        
    W, b = sess.run([W, b])

print(W,b)

#def main():
#    data_batch, label_batch = batch_generator([DATA_PATH])
#    generate_batches(data_batch, label_batch)

#if __name__ == "__main__":
#    main()


[[4.         0.23529412 0.2683706  0.11383499 0.75       0.42857143
  0.9302326  0.8333333  0.21875    0.38620284 0.         0.2310231
  0.35714287 0.33892617 0.65363127 0.         0.63860667 0.33333334
  0.         0.6666667  0.5        0.6666667  0.         0.6666667
  0.33333334 0.99502486 0.75       0.6014388  0.26086956 0.23076923
  0.         0.         0.         0.         0.         1.
  0.5       ]] [0.4996381]
[[2.         0.23529412 0.2172524  0.08739647 0.625      0.42857143
  0.93798447 0.8666667  0.10125    0.2865566  0.         0.20462047
  0.28696194 0.23106425 0.5375543  0.         0.48911467 0.33333334
  0.         0.6666667  0.5        0.5        0.         0.33333334
  0.33333334 0.9955224  0.5        0.4374101  0.         0.11538462
  0.         0.         0.         0.         0.         0.72727275
  0.5       ]] [0.4379656]
[[12.          0.          0.29073483  0.0821439   0.625       0.42857143
   0.9767442   0.95        0.19125     0.          0.          0.7